In [1]:
import re
import gensim
import string
import warnings
from collections import Counter
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import word_tokenize

from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

morph = MorphAnalyzer()
warnings.filterwarnings('ignore')

# Data

In [2]:
# Let's augment the stopword set with some Python syntax, numbers, two types of a long dash, and some random stopwords.
stops = set(stopwords.words('russian')) | {'gt', 'from', 'import', 'and', 'or', 'is', 'in',
                                           'if', 'for', 'while', 'none', 'null', 'return', 'yield',
                                           'break', 'pass', 'continue', 'int'} | \
                                            set(map(str, range(10))) | \
                                            {'–', '—', 'очень', 'n', 'a', 'x'}

def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def opt_normalize(texts, top=50000):
    unique = Counter()
    for text in texts:
        unique.update(text)
    
    norm_unique = {word: morph.parse(word)[0].normal_form for word, _ in unique.most_common(top)}
    
    norm_texts = []
    for text in texts:
        norm_words = [norm_unique.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    # For tokenization let's use the word_tokenize function from nltk.
    words = [word.strip(string.punctuation) for word in word_tokenize(text)]
    words = [word for word in words if word]
    
    return words

In [3]:
texts = open('habr_texts.txt').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts])

In [4]:
texts[0][:25]

['masstransit',
 'это',
 'open',
 'source',
 'библиотека',
 'разработать',
 'язык',
 'c',
 'net',
 'платформа',
 'работа',
 'шина',
 'дать',
 'который',
 'использоваться',
 'построение',
 'распределенный',
 'приложение',
 'реализация',
 'soa',
 'service',
 'oriented',
 'architecture',
 'качество',
 'message']

# n-grams

In [5]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.3)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

In [6]:
ngrammed_texts[0][:25]

['masstransit',
 'это',
 'open_source',
 'библиотека',
 'разработать',
 'язык_c',
 'net',
 'платформа',
 'работа',
 'шина',
 'дать',
 'который',
 'использоваться',
 'построение_распределенный',
 'приложение',
 'реализация',
 'soa',
 'service',
 'oriented',
 'architecture',
 'качество',
 'message',
 'мочь',
 'выступать',
 'rabbitmq']

Some meaningful n-grams have emerged: "open\_source", "язык\_C", "построение\_распределенный".

# LDA

In [7]:
dictionary = gensim.corpora.Dictionary(ngrammed_texts)

# Let's filter out frequent words a little less strictly.
dictionary.filter_extremes(no_above=0.5, no_below=20)
dictionary.compactify()

In [8]:
# Let's make sure we have deleted the stopwords.

assert all(stop not in dictionary.token2id for stop in stops)

In [9]:
print(dictionary)

Dictionary(9726 unique tokens: ['2-х', '3.0', 'address', 'api', 'architecture']...)


In [10]:
corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]

In [14]:
lda = gensim.models.LdaMulticore(corpus, num_topics=100, id2word=dictionary, passes=10)
topics = lda.print_topics()

The resulting model leaves much to be desired: only a few topics are actually interpretable.

In [18]:
# Servers.

topics[2]

(43,
 '0.038*"сервер" + 0.029*"т.е" + 0.017*"какой-то" + 0.016*"запрос" + 0.012*"делать" + 0.009*"очередь" + 0.009*"клиент" + 0.008*"либо" + 0.007*"ответ" + 0.006*"пользователь"')

In [22]:
# Cognitive science.

topics[6]

(42,
 '0.022*"человек" + 0.009*"год" + 0.005*"мозг" + 0.005*"исследование" + 0.005*"учёный" + 0.005*"пациент" + 0.004*"говорить" + 0.004*"результат" + 0.004*"«_»" + 0.004*"жизнь"')

In [24]:
# Programming courses?

topics[8]

(46,
 '0.011*"книга" + 0.010*"программа" + 0.009*"человек" + 0.009*"курс" + 0.007*"делать" + 0.007*"компьютер" + 0.006*"программирование" + 0.006*"написать" + 0.006*"читать" + 0.006*"писать"')

In [26]:
# Machine learning?

topics[10]

(59,
 '0.012*"алгоритм" + 0.012*"изображение" + 0.012*"модель" + 0.009*"сеть" + 0.009*"обучение" + 0.008*"результат" + 0.008*"метод" + 0.007*"задача" + 0.007*"нейросеть" + 0.006*"значение"')

In [15]:
lda.log_perplexity(corpus[:10000])

-8.234579989955254

# TF-IDF

In [28]:
tfidf = gensim.models.TfidfModel(dictionary=dictionary)
tfidf_corpus = tfidf[corpus]

In [41]:
tfidf_lda = gensim.models.LdaMulticore(tfidf_corpus, num_topics=100, id2word=dictionary, passes=25)
tfidf_topics = tfidf_lda.print_topics()

In [42]:
tfidf_lda.log_perplexity(tfidf_corpus[:10000])

-28.83737942599991

In [49]:
# Just random english words :)

tfidf_topics[7]

(53,
 '0.067*"lt_div" + 0.044*"let" + 0.023*"func" + 0.019*"def" + 0.019*"jsx" + 0.017*"react" + 0.017*"nil" + 0.016*"do" + 0.016*"sql_server" + 0.014*"view"')

In [52]:
# Frontend + space + some noise.

tfidf_topics[10]

(76,
 '0.058*"css" + 0.043*"react" + 0.042*"angular" + 0.031*"javascript" + 0.024*"роскомнадзор" + 0.017*"материя" + 0.013*"тёмный_материя" + 0.012*"redux" + 0.011*"svg" + 0.007*"переговоры"')

In [55]:
# Space + some noise.

tfidf_topics[13]

(87,
 '0.041*"марс" + 0.031*"миссия" + 0.030*"зонд" + 0.023*"планета" + 0.022*"аппарат" + 0.017*"автопилот" + 0.016*"земля" + 0.013*"посадка" + 0.013*"paypal" + 0.012*"солнечный"')

The increase in perplexion is in accordance with the overall feeling left after looking at the topics: they became even worse. In many cases a mix of several topics in one is evident, and in all the topics there is noise.

It's difficult to compare individual topics because for that one needs to establish an alignment between two sets of 100 topics — and this alignment probably doesn't exist.

During the preparation of this notebook I tried several different numbers of collection passes, and it appears that perplexion doesn't necessarily decrease with the increase of this parameter.

# NMF

In [24]:
stexts = [' '.join(text) for text in ngrammed_texts]
vectorizer = TfidfVectorizer(max_features=1000, min_df=40, max_df=0.5, ngram_range=(1, 3))
X = vectorizer.fit_transform(stexts)

In [25]:
model = NMF(n_components=20)
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=20, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [26]:
feat_names = vectorizer.get_feature_names()
top_words = model.components_.argsort()[:, :-11:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i+1, words)

1 ['делать', 'какой', 'что', 'какой то', 'ваш', 'книга', 'человек', 'что то', 'хотеть', 'знать']
2 ['функция', 'код', 'объект', 'значение', 'метод', 'класс', 'тип', 'алгоритм', 'строка', 'элемент']
3 ['игра', 'игрок', 'игровой', 'играть', 'разработчик', 'уровень', 'разработка', 'мир', 'карта', 'экран']
4 ['приложение', 'пользователь', 'android', 'разработчик', 'мобильный', 'разработка', 'сервис', 'api', 'google', 'платформа']
5 ['устройство', 'смартфон', 'датчик', 'функция', 'атака', 'производитель', 'безопасность', 'android', 'защита', 'драйвер']
6 ['сервер', 'клиент', 'запрос', 'сервис', 'дата центр', 'дата', 'центр', 'база_дать', 'облако', 'инфраструктура']
7 ['компания', 'сотрудник', 'год', 'рынок', 'бизнес', 'клиент', 'google', 'российский', 'microsoft', 'продажа']
8 ['камера', 'смартфон', 'звук', 'цена', 'экран', 'видео', 'модель', 'телефон', 'корпус', 'процессор']
9 ['файл', 'скрипт', 'настройка', 'пакет', 'папка', 'команда', 'строка', 'добавить', 'репозиторий', 'установка']
10 

Almost all of the topics seem nice and interpretable.
Only topics 1, 13, 15, and 20 are somewhat less clear. Besides, some of the topics intersect.

1: ?philosophy and literature
2: OOP
3: game development
4: web-development
5: gadgets
6: data infrastructure
7: IT business news
8: gadgets again
9: files, scripts, repositories
10: cognitive science
11: web
12: space
13: some programming languages
14: management in IT
15: some programming languages again
16: manufacturing technologies
17: papers and conferences
18: net
19: systems
20: ?frontend, web

# LDA with 20 topics (comparison)

In [27]:
lda = gensim.models.LdaMulticore(corpus, num_topics=20, id2word=dictionary, passes=10)
topics = lda.print_topics()

In [28]:
for i, topic in topics:
    print(i+1, re.findall(r'(?<=")[A-Za-zА-Яа-яЁё]+(?=")', topic))

1 ['человек', 'компания', 'год', 'делать', 'проект', 'говорить', 'деньга', 'сотрудник', 'хотеть']
2 ['lt', 'name', 'string', 'the', 'i', 'this', 'amp', 'файл', 'type']
3 ['запрос', 'сервер', 'система', 'диск', 'пользователь', 'проблема', 'тест', 'запись']
4 ['код', 'класс', 'объект', 'метод', 'файл', 'элемент', 'тип', 'функция', 'приложение', 'пример']
5 ['человек', 'год', 'компания', 'робот', 'система', 'машина', 'технология', 'автомобиль', 'ия', 'компьютер']
6 ['звук', 'человек', 'глаз', 'учёный', 'движение', 'тело', 'пациент', 'случай', 'система']
7 ['проект', 'задача', 'разработка', 'разработчик', 'сайт', 'делать', 'продукт', 'игра', 'команда', 'язык']
8 ['человек', 'год', 'ребёнок', 'ваш', 'исследование', 'мозг', 'курс', 'результат']
9 ['точка', 'объект', 'значение', 'алгоритм', 'вселенная', 'тип', 'число', 'результат', 'количество', 'координата']
10 ['игра', 'игрок', 'игровой', 'играть', 'процессор', 'сервер', 'система', 'уровень', 'видеокарта']
11 ['компания', 'пользователь', 'с

It's evident that almost all of the topics are noisy or outright uninterpretable.